
  # Homework 10

  First Last

  2020-04-23

In [1]:
from IPython import get_ipython
import numpy as np
import scipy.stats as stats
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns



In [2]:
# get_ipython().run_line_magic('matplotlib', 'widget')
# get_ipython().run_line_magic('matplotlib', 'inline')
# get_ipython().run_line_magic('config', "InlineBackend.figure_format = 'retina'")
# plt.rcParams['figure.figsize'] = (9, 5.5)
# mpl.rcParams['mathtext.fontset'] = 'stix'
# mpl.rcParams['font.size'] = 14
# mpl.rcParams['font.family'] = 'Times New Roman'




 ## Problem 1: Transformation of Random Variables

 Given that $X \sim \mathrm{Unif}(0, 1)$, let

 $$Y = \mathrm{logit}(X) \equiv \log \left( \frac{X}{1 - X} \right)$$

 Find the probability density function (PDF) of Y.


 <br/>
 <br/>

 ## Problem 2: Problem 7.1 on page 230 of _Computational Statistics_

 ---

 ### Problem 7.1 Metropolis-Hastings

 The goal of this problem is to investigate the role of the proposal
 distribution in a Metropolis-Hastings algorithm designed to simulate from the
 posterior distribution of a parameter $\delta$. In part (a), you are asked to
 simulate data from a distribution with $\delta$ known. For parts (b)-(d),
 provide an appropriate plot and a table summarizing the output of the
 algorithm. To facilitate comparisons, use the same number of iterations,
 random seed, starting values, and burn-in period for all implementations of
 the algorithm.

 * (a) Simulate 200 realizations from the mixture distribution in Equation
   (7.6) with $\delta = 0.7$. Draw a histogram of these data.

 > (Eq. 7.6):
 >
 > $\delta N(7, 0.5^2) + (1 - \delta) N(10, 0.5^2)$

 * (b) Implement an independence chain MCMC procedure to simulate from the
   posterior distribution of $\delta$, using your data from part (a)

 * (c) Implement a random walk chain with $\delta^* = \delta^{(t)} +
   \epsilon$ with $\epsilon \sim \mathrm{Unif}(-1, 1)$.

 * (d) Reparameterize the problem letting $U = \log \left[ \delta / (1 -
   \delta) \right]$ and $U^* = u^{(t)} + \epsilon$. Implement a random walk
   chain in $U$-space as in equation (7.8).

 > (Eq. 7.8):
 >
 > $\frac{f\left(\mathrm{logit}^{-1}\{u^*\}\right)\ | J(u^*) |\ g \left(u^{(t)} | u^*\right)}
   {f\left(\mathrm{logit}^{-1}\{u^{(t)}\}\right)\ | J(u^{(t)}) |\ g \left(u^* | u^{(t)}\right)}$

 * (e) Compare the estimates and convergence behavior of the three algorithms.



 <br/>
 <br/>

 ## Problem 3: Problem 7.2 on page 231 of _Computational Statistics_

 ---

 ### Problem 7.2 - Simulating realizations

 Simulating from the mixture distribution in Equation (7.6) is straightforward
 [see part (a) of Problem 7.1]. However, using the Metropolis-Hastings
 algorithm to simulate realizations from this distribution is useful for
 exploring the role of the proposal distribution.

 * (a) Implement a Metropolis-hastings algorithm to simulate from Equation
   (7.6) with $\delta = 0.7$, using $N(x^{(t)}, 0.01^2)$ as the proposal
   distribution. For each of three starting values, $x^{(0)} = 0,\ 7,
   $ and $15$, run the chain for 10,000 iterations. Plot the sample path
   of the output from each chain. If only one of the sample paths was
   available, what would you conclude about the chain? For each of the
   simulations, creat a histogram of the realizations with the true density
   superimposed on the historam. Based on your output from all three chains,
   what can you say about the behavior of the chain?

 > (Eq. 7.6):
 >
 > $\delta N(7, 0.5^2) + (1 - \delta) N(10, 0.5^2)$

 * (b) Now change the proposal distribution to imporove the convergence
   properties of the chain. using the new proposal distribution, repeat part
   (a).